## Ensemble methods base scikit-learn

基于scikit-learn，介绍一些ensemble的基础知识，也涉及部分代码。

ensemble methods（集成方法）的目标就是组合一些基预测器的结果，得到更为强健、泛化能力更强的模型。这种方法，算法的差异性很重要。

一共有两类这种方法：
1. __平均法（averaging methods）__：将一些__独立__的模型结果进行平均，得到的组合模型的效果一般会比单个模型效果更好, 例如：Bagging Methods 随机森林；
2. __提升法（boosting methods）__：基模型通过串行构建成一个最终模型，即$f(n)=g(f(n-1))$，从而减小组合分类器的bias。其目的就是将多个弱模型组合成一个强模型。例如：AdaBoost, Gradient tree boosting, XGBoost

### 平均法
#### Bagging method
Bagging method一般是将原始训练数据集随机抽样分成k个子集，然后训练多个模型，然后计算多个模型预测结果的平均值为最终决策（分类的话，选择阈值或类别最大个数）。对比Boost method，Bagging method更适合处理复杂模型（作为基模型），因为这个方法k而已降低过拟合，而Boost更适合选择弱模型作为基模型。

根据将原始训练数据集抽样分类的方法可以将Bagging Method分为以下几类：
1. __Pasting__: 随机抽样
2. __Bagging__: 有放回随机抽样
3. __Random Subspaces__: 随机抽取__特征__的子集
4. __Random Patches__: 同时随机抽取__特征__和样本

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import california_housing
from sklearn.ensemble import BaggingRegressor      # BaggingClassifier
from sklearn.tree import DecisionTreeRegressor     # DecisionTreeClassifier
from sklearn.metrics import mean_squared_error
bagging = BaggingRegressor(DecisionTreeRegressor(), max_samples=0.5, max_features=0.5)
datas = california_housing.fetch_california_housing()
X = datas['data']
y = datas['target']
indices = np.random.permutation(X.shape[0])
i_test = indices[-100:]
i_train = indices[:-10000]
X_test = X[i_test]
y_test = y[i_test]
X_train = X[i_train]
y_train = y[i_train]
b_estimator = bagging.fit(X_train, y_train)
b_predict = b_estimator.predict(X_test)
b_error = mean_squared_error(y_test, b_predict)
d_estimator = DecisionTreeRegressor()
d_estimator.fit(X_train, y_train)
d_predict = d_estimator.predict(X_test)
d_error = mean_squared_error(y_test, d_predict)
print("Mean Squared Error:")
print("BaggingRegressor:{error:.4f}".format(error=b_error))
print("DecisionTreeRegressor:{error:.4f}".format(error=d_error))

Mean Squared Error:
BaggingRegressor:0.3678
DecisionTreeRegressor:0.9102


#### Forests of randomized trees

在`sklearn.ensemble`包中，包括两个基于随机决策树的average method：the RandomForest和Extra-Trees method。

##### Random Forests
ensemble包的随机森林里面的训练样本是有放回采样（bootstrap），并且每个树的结点在split时不是选择所有特征最优的进行split，而是从feature的集合中随机抽取一个子集，然后取子集中最优的特征进行split，这样会导致单个树的偏差较高，但是平均之后降低的variance会抵消这部分增加的偏差。而随机森林的每棵树的split的。

_ Q: max_features:这个是每个树的最大特征还是每次split的时候最大特征（明显是每棵树，因为这个参数作为每棵树（DecisionTree）的参数传入）；还有就是这个特征是有放回还是无放回的随机抽样？_

In [2]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor()
rfr.fit(X_train, y_train)
y_predict = rfr.predict(X_test)
error = mean_squared_error(y_test, y_predict)
print("Mean Squared Error:")
print("RandomForestRegressor:{error:.4f}".format(error=b_error))

Mean Squared Error:
RandomForestRegressor:0.3678


##### Extremely Randomized Trees

和随机森林相同的一点就是，也是随机抽取一部分特征，然后对这些特征随机产生split的阈值，然后从这些阈值中选择一个最好的split。

In [3]:
from sklearn.ensemble import ExtraTreesRegressor
clf = ExtraTreesRegressor(n_estimators=10, max_depth=None, min_samples_split=3, random_state=0)
clf.fit(X_train, y_train)
y_predict = clf.predict(X_test)
error = mean_squared_error(y_test, y_predict)
print("Mean Squared Error:")
print("ExtraTreesRegressor:{error:.4f}".format(error=b_error))

Mean Squared Error:
ExtraTreesRegressor:0.3678


##### Totally Random Trees Embedding
一种无监督的数据转换方式，使用完全随机树森林，来将低维数据转换到高维，以求得更好的进行分类应用。其采用的是one-hot（one-of-K）编码方式，新的特征空间中，每一个特征相当于一个叶节点的标识位（1标识在这个叶节点，0标识不在）。

所以最终编码的长度为：
$$n\_estimators * 2^{max\_depth}$$

#### Boosting方法

下面介绍几个`sklearn.ensemble`中包含的几个boost算法

##### AdaBoost

AdaBoost的核心是迭代学习一系列弱学习器，每次迭代时都对数据进行修改，而最终结构是所有弱分类器的加权平均和。第一次每个训练数据的权值为$w_i=1/N$, N是训练样本数，也就是说最初所有样本等权，每次迭代时，增加误分类样本的权值而降低正确分类样本的权值。样本的权值会影响到下一个模型在评判训练误差时的计算，从而影响模型效果。

In [4]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import cross_val_score
clf = AdaBoostRegressor(n_estimators=101)
clf.fit(X_train, y_train)
y_predict = clf.predict(X_test)
error = mean_squared_error(y_test, y_predict)
print("Mean Squared Error:")
print("AdaBoostRegressor:{error:.4f}".format(error=b_error))

Mean Squared Error:
AdaBoostRegressor:0.3678


#### Gradient Tree Boosting

Gradient Tree Boosting或者Gradient Boosted Regression Trees(GBRT)是一个可以推广到任意损失函数的Boost方法，即就是在计算下一次迭代所要新加入的函数时使用的损失函数不限制。
GBRT的优点：
* 适合使用混合类型数据（各种类型的特征，例如：标量、数值）
* 输出空间的鲁棒性（可以通过调整损失函数来拟合各种类型的输出）

缺点：
* 可扩展性，由于Boost方法都是顺序计算，所以比较难进行并行化计算

`sklearn.ensemble`提供了使用GBRT完成的分类和回归方法:GradientBoostingClassifier/GradientBoostingRegressor，下面只介绍回归，分类类似（因为上面的数据集y值是连续变量），可以通过`set_params`来热启动，从原有的clf中增加弱模型来得到新的模型（additive models）

##### 回归

In [5]:
from sklearn.ensemble import GradientBoostingRegressor
clf = GradientBoostingRegressor(n_estimators=100, learning_rate=1.0, max_depth=2, random_state=0)
clf.fit(X_train, y_train)
y_predict = clf.predict(X_test)
b_error = mean_squared_error(y_test, y_predict)
print("Mean Squared Error:")
print("GradientBoostingClassifier:{error:.4f}".format(error=b_error))

Mean Squared Error:
GradientBoostingClassifier:0.5756


In [6]:
iters_error = clf.train_score_    # 在每个迭代中训练误差的值

In [7]:
clf.staged_predict(X_test)     # staged_predict 可以获得X_test每一个测试样例在每个阶段（每次boost新增弱模型时）的测试误差

<generator object GradientBoostingRegressor.staged_predict at 0x103d9dd00>

一些数学推导：

GBRT作为一个加法模型，有如下形式：
$$
F(x)=\sum_{m=1}^M\gamma_mh_m(x)\\
F_m(x)=F_{m-1}(x) + \gamma_mh_m(x)\\
F_m(x)=F_{m_1}(x) + arg\min_h\sum_{i=1}^nL(y_i, F_{m-1}(x_i) + h(x))
$$
$F_0$比较特殊，一般会选择目标值的均值